In [ ]:
import pandas as pd 
import json,os
# location = 'cmip6_variables.xlsx'
location = 'BrandingofCMIP6Variables240905.xlsx'

In [ ]:
# import pprint
# pprint.pprint(json_data)

In [ ]:
import openpyxl

# Load the Excel workbook
workbook = openpyxl.load_workbook(location, data_only=True)
print(workbook.active)

# Choose a specific sheet
sheet_name = "CMIP6_branded_variables"
sheet = workbook[sheet_name]


In [ ]:
def column_index_to_letter(index):
    """
    Convert a 0-based column index to its corresponding letter.
    For example: 0 -> A, 1 -> B, 2 -> C, ..., 25 -> Z, 26 -> AA, 27 -> AB, ...
    """
    letters = ''
    while index >= 0:
        letters = chr(index % 26 + ord('A')) + letters
        index = index // 26 - 1
    return letters

def get_column_letters(limit):
    """
    Get a list of column letters up to the given limit.
    For example, if limit is 25, returns ['A', 'B', ..., 'Z'].
    """
    columns = []
    for i in range(limit + 1):
        column_letter = column_index_to_letter(i)
        columns.append([column_letter,i])
    return dict(columns)

collist = get_column_letters(len(list(sheet.columns)))



In [ ]:
# Map key names to column letters
mapping = {
    "standard_name": "P",
    "units": "F",
    "type": "AN",
    "positive": "AM",
    "variable_root_name": "BY",
    "definition_qualifier": "CR",
    "branding_suffix": "HL",
    "brand_description": "CT",
    "dimensions": "AZ",
    "cell_methods": "T",
    "temporalLabelDD": "BE",
    "verticalLabelDD": "AV",
    "horizontalLabelDD": "AW",
    "areaLabelDD": "AA",
    "cell_measures": "BF",
    "samplingIntervals": "BD",
    "regions": "CV",
    "mip_table": "A",
    "table_entry": "BA",
    "out_name": "BG",
    "questions_and_notes": "H",
    # "specific_issues": "GX:HI",
        'specific issues 1': 'GX',
    'specific issues 2': 'GY',
    'specific issues 3': 'GZ',
    'specific issues 4': 'HA',
    'specific issues 5': 'HB',
    'specific issues 6': 'HC',
    'specific issues 7': 'HD',
    'specific issues 8': 'HE',
    'specific issues 9': 'HF',
    'specific issues 10': 'HG',
    'specific issues 11': 'HH',
    'specific issues 12': 'HI',
    "contributing_institutions": "M",
    "contributing_models": "N",
    "frequency": "BC",
    "region": "CV",
    "modeling_realm": "BB",
    "long_name": "D",
    "comment": "E"
}


    
with open("../mapping_output.txt", "w") as file:
        for c, i in mapping.items():
            idx = collist[i.strip()]
            output_line = f"{c} - {i} - {collist[i]} - {list(sheet.columns)[idx][0].value}\n"
            file.write(output_line)
            print(output_line, end='')




In [ ]:

all_json = []

base = '../DReq'
write = True

# with open("{base}/frame.jsonld", "w") as f:
#     frame = {'@prefix': 'dr:'}
#     f.write(json.dumps(frame, indent=4))


# Iterate over the rowscollist of the dataframe
for rn,row in enumerate(sheet.iter_rows(min_row=2, values_only=True)):
    
    if rn>2062 -1:
        break
    
    # if rn == 0:
    #     continue
        
    id = row[collist['BI']].lower().replace('_', '-')
        
    provenance = {  # possibly multiple phases per branded name, each containing
                                "@id": f'dr:cmip6/{id}',
                                "@type": "dr-variable-cmip6",
                                
                                "almost-unique-identifier": row[collist['BH']],
                                "mip_table": [row[collist['A']]],  # possibly multiple tables listed, each containing
                                "table_entry": row[collist['BA']],
                                "out_name": row[collist['BG']],
                                "questions_and_notes": row[collist['H']] if not pd.isna(row[collist['H']]) else "",
                                # "specific_issues": dict([[row[r],r] for r in range(collist['GX'], collist['HI']+1) if not pd.isna(row[r])]),  # list all cells not blank
                                "specific_issues": [row[r] for r in range(collist['GX'], collist['HI']+1) if not pd.isna(row[r])],  # list all cells not blank
                                "contributing_institutions": row[collist['M']],
                                "contributing_models": row[collist['N']],
                                "frequency": row[collist['BC']],
                                "region": row[collist['CV']],
                                "modeling_realm": [realm.strip() for realm in str(row[collist['BB']]).split(',')] if not pd.isna(row[collist['BB']]) else [],  # a list
                                "dimensions": row[collist['AX']] if not pd.isna(row[collist['AX']]) else row[collist['AZ']],  # if included, this will override the above proposed new dimensions to recover CMIP6 specification
                                "cell_methods": row[collist['Q']] if not pd.isna(row[collist['Q']]) else row[collist['T']],  # if included, this will override the above proposed new cell_methods to recover CMIP6 specification
                                "long_name": row[collist['D']],
                                "comment": row[collist['E']]
                            }
    if write:
        with open(f"{base}/cmip6/{id}.json", "w") as f:
            f.write(json.dumps(provenance, indent=4)) 
        
        
    json_obj = {
        
        "@id": f'dr:variables/{id}',
        "@type": "dr-variable",
        
        
        "standard_name": row[collist['P']],  # multiple standard names listed, each containing
        "variable_root_name":row[collist['BY']],
        "units": row[collist['F']],
        "type": row[collist['AN']],  # consider replacing "real", "integer", etc. with netCDF type indicators
        "positive": row[collist['AM']],
        "sn_status": "approved",
        
        "branding": {
            "temporalLabelDD": row[collist['BE']],
            "verticalLabelDD": row[collist['AV']],
            "horizontalLabelDD": row[collist['AW']],
            "areaLabelDD": row[collist['AA']]
        },
        
        "definition_qualifier": row[collist['CR']] if not pd.isna(row[collist['CR']]) else "none",  # if cell is blank, enter "none"

        "brand_description": row[collist['CT']],
        "bn_status": "accepted",
        "dimensions": row[collist['AX']].split(' ') if not pd.isna(row[collist['AX']]) else row[collist['AX']].split(' '),  # for CMIP6 this might be overridden by a different value
        "cell_methods": row[collist['Q']] if not pd.isna(row[collist['Q']]) else row[collist['Q']],  # for CMIP6 this might be overridden by a different value
        
        "cell_measures": row[collist['BF']],
        "samplingIntervals": [row[collist['BD']]],  # initially list all frequencies from CMIP6 that apply to the branded variable
        "regions": [row[collist['CV']]],  # initially list all regions from CMIP6 that apply to the branded variable
        # "history": "2023-11-10: registered (from CMIP6 tables, possibly modified/corrected)",
        "used_in": {
            "cmip6": {"@id":f'dr:cmip6/{id}'}
        }
    }
    
    if write:
        with open(f"{base}/variables/{id}.json", "w") as f:
            f.write(json.dumps(json_obj, indent=4))
            
        



specific issues 1 - GX - 205 - Modify dimensions.  No need for special horizontal coordinates. 
specific issues 2 - GY - 206 - Modify dimensions.  Add or remove scalar coordinate.
specific issues 3 - GZ - 207 - Modify dimension.  Other
specific issues 4 - HA - 208 - Check units.
specific issues 5 - HB - 209 - Standard name issue.
specific issues 6 - HC - 210 - Modiy root name.
specific issues 7 - HD - 211 - Deprecate? Eliminate one of "point" or "mean" variable options.
specific issues 8 - HE - 212 - Depricate?  Few institutions contributed variable to CMIP6.
specific issues 9 - HF - 213 - Eliminate or deprecate?  
specific issues 10 - HG - 214 - Modify cell methods.  Include "where air".
specific issues 11 - HH - 215 - Modify cell methods.  Include "where sea".
specific issues 12 - HI - 216 - Modify cell methods.



bi branded variable BI
proposed branded.  BK

cmip6 amlost unique name BH 


dims AX 
proposed dims AZ

cellmethods Q 
proposed R

In [ ]:
# DIMENSIONS> 

dims = 'GX GY GZ'

for i in dims.split(' '):
    
    j = collist[i]
    kn = list(sheet.columns)[j][0].value.replace('"',"'")
    knb = kn.replace(' ','_').replace('.','').replace('(','').replace(')','').replace('/','_')
    
    print(kn)
    if kn != None:
        
        
        print(id, i,j,kn)

        new = 'AZ'
            
            
        try:
            os.popen('git branch -D '+knb).read()
        except:
            ...
            
        try:
            print(os.popen(f"git checkout -b {knb}").read())
        except: 
            print(os.popen(f"git checkout {knb}").read())
            
            
        os.popen(f'git pull').read()
        for rn,row in enumerate(sheet.iter_rows(min_row=2, values_only=True)):
            
            
            k = row[j]
            if k == None:
                continue
            
            if rn>2062 - 1:
                break
                
            id = row[collist['BI']].lower().replace('_', '-')
            
            # print(row[collist[new]])
            
            if row[collist[new]] != None:
                
                js = json.load(open(f"{base}/variables/{id}.json"))
                js['dimensions'] = row[collist[new]]
                json.dump(js, open(f"{base}/variables/{id}.json", "w"), indent=4)
            
                js = json.load(open(f"{base}/cmip6/{id}.json"))
                js['dimensions'] = row[collist[new]]
                
                if k in js['specific_issues']:
                    js['specific_issues'].remove(k)
                json.dump(js, open(f"{base}/cmip6/{id}.json", "w"), indent=4)
            
        
        os.popen('git add -A').read()
        print(os.popen(f'git commit -m "Corrected dimensions for {kn}"').read())
        print(os.popen(f"git push origin {knb} --force").read())
        print(os.popen(f'gh pr create --title "{kn}" --body "An automated correction suggestion by @taylor13 "').read())
        os.popen('git checkout main').read()

In [ ]:
# CELL METHODS

cell_methods = 'HG HH HI'

for i in cell_methods.split(' '):
    
        
    # i = 'HG'
    j = collist[i]
    kn = list(sheet.columns)[j][0].value.replace('"',"'")
    knb = kn.replace(' ','_').replace('.','').replace('(','').replace(')','').replace('/','_')
    
    
    print(kn)
    
    if kn != None:
        
        
        print(id, i,j,kn)

        new = 'R'
            
            
        try:
            os.popen('git branch -D '+knb).read()
        except:
            ...
            
        try:
            print(os.popen(f"git checkout -b {knb}").read())
        except: 
            print(os.popen(f"git checkout {knb}").read())
            
            
        os.popen(f'git pull').read()
        for rn,row in enumerate(sheet.iter_rows(min_row=2, values_only=True)):
            
            
            k = row[j]
            if k == None:
                continue
            
            if rn>2062 - 1:
                break
                
            id = row[collist['BI']].lower().replace('_', '-')
            
            # print(row[collist[new]])
            
            if row[collist[new]] != None:
                
                js = json.load(open(f"{base}/variables/{id}.json"))
                js['cell_methods'] = row[collist[new]]
                json.dump(js, open(f"{base}/variables/{id}.json", "w"), indent=4)
            
                js = json.load(open(f"{base}/cmip6/{id}.json"))
                js['cell_methods'] = row[collist[new]]
                
                if k in js['specific_issues']:
                    js['specific_issues'].remove(k)
                json.dump(js, open(f"{base}/cmip6/{id}.json", "w"), indent=4)
            
        
        os.popen('git add -A').read()
        print(os.popen(f'git commit -m "Corrected cell_methods for {kn}"').read())
        print(os.popen(f"git push origin {knb} --force").read())
        print(os.popen(f'gh pr create --title "{kn}" --body "An automated correction suggestion by @taylor13 "').read())
        os.popen('git checkout main').read()

In [ ]:

# all_json = []

# # Iterate over the rowscollist[ of the dataframe
# for rn,row in enumerate(sheet.iter_rows(min_row=2, values_only=True)):
    
#     if rn>2062:
#         break
        
#     json_obj = {
#         "standard_name": row[collist['P']],  # multiple standard names listed, each containing
#         "units": row[collist['F']],
#         "type": row[collist['AN']],  # consider replacing "real", "integer", etc. with netCDF type indicators
#         "positive": row[collist['AM']],
#         "sn_status": "approved",
#         "variable_root_name": {
#             row[collist['BY']]: {  # possibly multiple root names per standard name, each containing
#                 "definition_qualifier": row[collist['CR']] if not pd.isna(row[collist['CR']]) else "none",  # if cell is blank, enter "none"
#                 "branding_suffix": {
#                     row[collist['HL']]: {  # possibly multiple branding suffixes per root, each containing
#                         "brand_description": row[collist['CT']],
#                         "bn_status": "accepted",
#                         "dimensions": row[collist['AZ']].split(' ') if not pd.isna(row[collist['AZ']]) else row[collist['AX']].split(' '),  # for CMIP6 this might be overridden by a different value
#                         "cell_methods": row[collist['T']] if not pd.isna(row[collist['T']]) else row[collist['Q']],  # for CMIP6 this might be overridden by a different value
#                         "temporalLabelDD": row[collist['BE']],
#                         "verticalLabelDD": row[collist['AV']],
#                         "horizontalLabelDD": row[collist['AW']],
#                         "areaLabelDD": row[collist['AA']],
#                         "cell_measures": row[collist['BF']],
#                         "samplingIntervals": [row[collist['BD']]],  # initially list all frequencies from CMIP6 that apply to the branded variable
#                         "regions": [row[collist['CV']]],  # initially list all regions from CMIP6 that apply to the branded variable
#                         "history": "2023-11-10: registered (from CMIP6 tables, possibly modified/corrected)",
#                         "used_in": {
#                             "CMIP6": {  # possibly multiple phases per branded name, each containing
#                                 "mip_table": [row[collist['A']]],  # possibly multiple tables listed, each containing
#                                 "table_entry": row[collist['BA']],
#                                 "out_name": row[collist['BG']],
#                                 "questions_and_notes": row[collist['H']] if not pd.isna(row[collist['H']]) else "",
#                                 # "specific_issues": dict([[row[r],r] for r in range(collist['GX'], collist['HI']+1) if not pd.isna(row[r])]),  # list all cells not blank
#                                 "specific_issues": [row[r] for r in range(collist['GX'], collist['HI']+1) if not pd.isna(row[r])],  # list all cells not blank
#                                 "contributing_institutions": row[collist['M']],
#                                 "contributing_models": row[collist['N']],
#                                 "frequency": row[collist['BC']],
#                                 "region": row[collist['CV']],
#                                 "modeling_realm": [realm.strip() for realm in str(row[collist['BB']]).split(',')] if not pd.isna(row[collist['BB']]) else [],  # a list
#                                 "dimensions": row[collist['AX']] if not pd.isna(row[collist['AX']]) else row[collist['AZ']],  # if included, this will override the above proposed new dimensions to recover CMIP6 specification
#                                 "cell_methods": row[collist['Q']] if not pd.isna(row[collist['Q']]) else row[collist['T']],  # if included, this will override the above proposed new cell_methods to recover CMIP6 specification
#                                 "long_name": row[collist['D']],
#                                 "comment": row[collist['E']]
#                             }
#                         }
#                     }
#                 }
#             }
#         }
#     }
    
#     all_json.append(json_obj)

In [ ]:
len(all_json)  # 2062

In [ ]:
# stringify = json.dumps(all_json, indent=4)
# print(stringify)

In [ ]:
# for i in all_json:
#     # 
#     print(len(i['variable_root_name']))

In [ ]:
# with open('MIP_variables_KT.json', 'w') as json_file:
#     json.dump(all_json, json_file, indent=4)